In [2]:
import numpy as np
import classy as cl
from classy import Class
import matplotlib.pyplot as plt
import pickle
#import derivatives as dv
pi=np.pi

from scipy.integrate import quad
from scipy import integrate
import time
from scipy.interpolate import interp1d
hbar=6.58*10**(-16)  # eV s units
eq=0.302 # electric charge
alpha=eq**2/(4*np.pi)
Mpl=2.435*10**(18)
c=2.998*10**5 #km/s
ccm=2.998*10**(10) #cm/s

############ Rewritten to be in h / Mpc Units #####################



h=.678
OmegaL=0.692
OmegaM=0.308
H100=100/(3.086*10**(19.))*hbar  # 100 km/s/Mpc *(1 Mpc/3.086*10^19 km) * hbar(s eV) -> eV
rhoc = 3*((Mpl*10**(9))**2)*(h**2)*(H100**2) #in eV^(4), need to convert Mpl in GeV to eV
kB= 8.617*10**(-5) #boltzmann's constant in eV / K
Tgamma=2.725*kB
rhogamma= 2*(np.pi**2)*(Tgamma**4)/(30.)
OmegaR=rhogamma/rhoc

Stepsize = {
    'A_s': 0.1e-9,
    'n_s': 0.01,
    '100*theta_s': 0.005,
    'omega_b': 0.0008,
    'omega_cdm': 0.003,
    'tau_reio':0.02,
    'N_ur':0.03}

def H(a):
    return h*H100*np.sqrt(OmegaR*a**(-4) + OmegaM*a**(-3)+OmegaL)/hbar
def HMpc(a):
    return h*100*np.sqrt(OmegaR*a**(-4) + OmegaM*a**(-3)+OmegaL)  # returns H(a)  in km/s / Mpc

Spheredeg2=360**2/pi

def ztoa(z):
    return 1./(1.+z)

def distanceMpc(z):
    temp=quad(lambda x:1/(x*x*HMpc(x)),ztoa(z),1)[0]
    return temp*3*10.**(5.)*h
#return temp*3*10.**(5.)

def shell(z,Deltaz):
    temp=quad(lambda x:1/(x*x*HMpc(x)),ztoa(z+Deltaz),ztoa(z))[0]
    return temp*3*10.**(5.)*h
#return temp*3*10.**(5.)

def shellV(z,Deltaz,deg2):
    return 4*pi*shell(z,Deltaz)*distanceMpc(z)**2*deg2/Spheredeg2

def Dt(z):
    temp1=quad(lambda x:1/((x*HMpc(x)/100.)**3),0.0001,ztoa(z))[0]
    temp2=quad(lambda x:1/((x*HMpc(x)/100.)**3),0.0001,1)[0]
    return (temp1/temp2)*(HMpc(ztoa(z))/HMpc(1.))

def Dta(a):
    temp1=quad(lambda x:1/((x*HMpc(x)/100.)**3),0.0001,a)[0]
    temp2=quad(lambda x:1/((x*HMpc(x)/100.)**3),0.0001,1)[0]
    return (temp1/temp2)*(HMpc(a)/HMpc(1.))


def ng_z_bins(deg2,zmax,Nobs,z_space=0.1):
    z_bins=np.arange(z_space,zmax,z_space)
    ng_bins={}
    vol_tot=0
    for z in z_bins:
        vol_tot+=shellV(z,z_space,deg2)
    for z_j in z_bins:
        ng_bins[z_j]=Nobs/vol_tot
    return z_bins,ng_bins,vol_to